# Aula 6 - ALS

In [1]:
!pip install pyspark

In [2]:
!pip install findspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row # formato que ALS trabalha

In [4]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [11]:
import requests
import pandas as pd
import io

url = "https://raw.githubusercontent.com/SandraRojasZ/Pos_Tech_Data_Analytics/main/Base_de_Dados/sample_movielens_ratings.txt"
# lines = spark.read.text("sample_movielens_ratings.txt").rdd

# Download the data from the URL
response = requests.get(url)
response.raise_for_status()  # Raise an exception if the request fails

# Load the data into an RDD
rdd = spark.sparkContext.parallelize(response.text.splitlines())

# Process each line to separate values
rdd_split = rdd.map(lambda line: line.split("::"))

In [14]:
ratingsRDD = rdd_split.map(lambda p: Row(userId=int(p[0]), \
                                     movieId=int(p[1]), \
                                     rating=float(p[2]), \
                                     timestamp=int(p[3])
                                     ))

In [15]:
ratings = spark.createDataFrame(ratingsRDD)

In [19]:
rdd.collect()

['0::2::3::1424380312',
 '0::3::1::1424380312',
 '0::5::2::1424380312',
 '0::9::4::1424380312',
 '0::11::1::1424380312',
 '0::12::2::1424380312',
 '0::15::1::1424380312',
 '0::17::1::1424380312',
 '0::19::1::1424380312',
 '0::21::1::1424380312',
 '0::23::1::1424380312',
 '0::26::3::1424380312',
 '0::27::1::1424380312',
 '0::28::1::1424380312',
 '0::29::1::1424380312',
 '0::30::1::1424380312',
 '0::31::1::1424380312',
 '0::34::1::1424380312',
 '0::37::1::1424380312',
 '0::41::2::1424380312',
 '0::44::1::1424380312',
 '0::45::2::1424380312',
 '0::46::1::1424380312',
 '0::47::1::1424380312',
 '0::48::1::1424380312',
 '0::50::1::1424380312',
 '0::51::1::1424380312',
 '0::54::1::1424380312',
 '0::55::1::1424380312',
 '0::59::2::1424380312',
 '0::61::2::1424380312',
 '0::64::1::1424380312',
 '0::67::1::1424380312',
 '0::68::1::1424380312',
 '0::69::1::1424380312',
 '0::71::1::1424380312',
 '0::72::1::1424380312',
 '0::77::2::1424380312',
 '0::79::1::1424380312',
 '0::83::1::1424380312',
 '0:

In [18]:
# Como o dado está no arquivo
rdd_split.collect()

[['0', '2', '3', '1424380312'],
 ['0', '3', '1', '1424380312'],
 ['0', '5', '2', '1424380312'],
 ['0', '9', '4', '1424380312'],
 ['0', '11', '1', '1424380312'],
 ['0', '12', '2', '1424380312'],
 ['0', '15', '1', '1424380312'],
 ['0', '17', '1', '1424380312'],
 ['0', '19', '1', '1424380312'],
 ['0', '21', '1', '1424380312'],
 ['0', '23', '1', '1424380312'],
 ['0', '26', '3', '1424380312'],
 ['0', '27', '1', '1424380312'],
 ['0', '28', '1', '1424380312'],
 ['0', '29', '1', '1424380312'],
 ['0', '30', '1', '1424380312'],
 ['0', '31', '1', '1424380312'],
 ['0', '34', '1', '1424380312'],
 ['0', '37', '1', '1424380312'],
 ['0', '41', '2', '1424380312'],
 ['0', '44', '1', '1424380312'],
 ['0', '45', '2', '1424380312'],
 ['0', '46', '1', '1424380312'],
 ['0', '47', '1', '1424380312'],
 ['0', '48', '1', '1424380312'],
 ['0', '50', '1', '1424380312'],
 ['0', '51', '1', '1424380312'],
 ['0', '54', '1', '1424380312'],
 ['0', '55', '1', '1424380312'],
 ['0', '59', '2', '1424380312'],
 ['0', '61', '

In [17]:
ratings.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     0|      2|   3.0|1424380312|
|     0|      3|   1.0|1424380312|
|     0|      5|   2.0|1424380312|
|     0|      9|   4.0|1424380312|
|     0|     11|   1.0|1424380312|
|     0|     12|   2.0|1424380312|
|     0|     15|   1.0|1424380312|
|     0|     17|   1.0|1424380312|
|     0|     19|   1.0|1424380312|
|     0|     21|   1.0|1424380312|
|     0|     23|   1.0|1424380312|
|     0|     26|   3.0|1424380312|
|     0|     27|   1.0|1424380312|
|     0|     28|   1.0|1424380312|
|     0|     29|   1.0|1424380312|
|     0|     30|   1.0|1424380312|
|     0|     31|   1.0|1424380312|
|     0|     34|   1.0|1424380312|
|     0|     37|   1.0|1424380312|
|     0|     41|   2.0|1424380312|
+------+-------+------+----------+
only showing top 20 rows



In [20]:
(training, test) = ratings.randomSplit([0.8, 0.2])

In [21]:
als = ALS(maxIter=5, \
          regParam=0.01, \
          userCol = "userId", \
          itemCol = "movieId", \
          ratingCol = "rating", \
          coldStartStrategy="drop"
          )

In [22]:
model = als.fit(training)

In [24]:
# Previsões
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Erro médio quadrático = " + str(rmse))

Erro médio quadrático = 1.8947580604414491


In [26]:
# se pega 10 recomendações
userRec = model.recommendForAllUsers(10)

In [28]:
userRec.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{46, 4.9922953},...|
|    10|[{27, 5.207564}, ...|
|     0|[{2, 4.9741974}, ...|
|     1|[{22, 4.226261}, ...|
|    21|[{29, 4.6964297},...|
|    11|[{7, 5.63011}, {1...|
|    12|[{90, 5.788821}, ...|
|    22|[{83, 5.409505}, ...|
|     2|[{27, 5.564542}, ...|
|    13|[{30, 3.3628783},...|
|     3|[{51, 5.042538}, ...|
|    23|[{32, 5.3649983},...|
|     4|[{2, 4.536874}, {...|
|    24|[{74, 5.396768}, ...|
|    14|[{29, 5.1305885},...|
|     5|[{46, 4.6487126},...|
|    15|[{46, 4.7689366},...|
|    25|[{8, 5.559787}, {...|
|    26|[{90, 5.956302}, ...|
|     6|[{25, 4.8275266},...|
+------+--------------------+
only showing top 20 rows



In [29]:
movieRecs = model.recommendForAllItems(10)

In [30]:
movieRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     20|[{22, 1.2246422},...|
|     40|[{2, 4.443407}, {...|
|     10|[{23, 3.515244}, ...|
|     50|[{12, 3.7478998},...|
|     80|[{26, 3.30585}, {...|
|     70|[{10, 4.619445}, ...|
|     60|[{25, 3.9291804},...|
|     90|[{26, 5.956302}, ...|
|     30|[{26, 5.3152003},...|
|      0|[{18, 3.595374}, ...|
|     31|[{12, 3.886595}, ...|
|     81|[{28, 4.8293657},...|
|     91|[{28, 4.5408816},...|
|      1|[{15, 3.5614436},...|
|     41|[{4, 3.5793457}, ...|
|     61|[{24, 3.4341395},...|
|     51|[{3, 5.042538}, {...|
|     21|[{26, 3.1432195},...|
|     11|[{18, 3.8664901},...|
|     71|[{18, 3.898484}, ...|
+-------+--------------------+
only showing top 20 rows



In [31]:
users = ratings.select(als.getUserCol()).distinct()

In [32]:
users.show()

+------+
|userId|
+------+
|     0|
|     7|
|     6|
|     9|
|     5|
|     1|
|    10|
|     3|
|    12|
|     8|
|    11|
|     2|
|     4|
|    13|
|    14|
|    26|
|    29|
|    19|
|    22|
|    25|
+------+
only showing top 20 rows



In [33]:
# Recomendações
UserRecsOnlyItemId = userRec.select(userRec['userId'], \
                                    userRec['recommendations']['movieid']
                                    )

In [34]:
UserRecsOnlyItemId.show(10, False)

+------+----------------------------------------+
|userId|recommendations.movieid                 |
+------+----------------------------------------+
|20    |[46, 22, 62, 98, 94, 30, 77, 74, 52, 83]|
|10    |[27, 87, 83, 70, 30, 32, 2, 40, 49, 42] |
|0     |[2, 92, 90, 9, 52, 62, 91, 63, 49, 81]  |
|1     |[22, 46, 62, 95, 55, 83, 98, 77, 85, 17]|
|21    |[29, 53, 75, 58, 18, 72, 74, 69, 30, 96]|
|11    |[7, 18, 32, 79, 30, 23, 48, 63, 49, 69] |
|12    |[90, 68, 27, 64, 17, 36, 35, 32, 94, 31]|
|22    |[83, 30, 88, 22, 74, 51, 75, 52, 94, 62]|
|2     |[27, 8, 83, 47, 91, 39, 40, 34, 89, 2]  |
|13    |[30, 29, 58, 18, 53, 69, 75, 74, 63, 52]|
+------+----------------------------------------+
only showing top 10 rows

